In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.image as mpimg

In [ ]:
#Part 1: The Plus Minus Method analysis.
#You will need the survey data to run this part of the code. The data can be requested from Geoscience Australia through https://dx.doi.org/10.26186/146309.

In [ ]:
import re #Only needed for the first part of the code.

In [ ]:
#Part 1.1
#All files
files = ['74-WAB', '74-WAC', '76-WBA', '79-WDA', '79-WDB', '79-WDF', '79-WDG', '80-WFJ', '80-WFW', '80-WGE', '80-WGG-WGT', '80-WGK-82-WLN', '80-WGN', '80-WGS', '80-WHR', '82-QNB', '82-RAP', '82-RAR2', '82-RAX', '82-WLA', '82-WLE', '84-TZH', '84-TZJ', '84-TZL', '84-TZM', '84-TZP', '84-TZW', '84-TZX', '84-TZZ', '84-WMD', '84-WME', '84-WMJ', '84-WML', '84-WMM', '84-WMQ', '84-WMT', '84-WMX', '84-WMZ', '84-WNA', '84-XAA', '84-XAB', '84-XAF', '84-XAH', '84-XEZ', '84-XFG', '85-WPC', '85-WPD', '85-WPF', '85-WPY', '85-YGN', '85-YGS', '85-YGW', '85-YRS', '86-ADS', '86-AEB', '86-AED', '86-AEY', '87-WRA', '87NT-05', '87NT-07', '87NT-12', '87NT-19', '88-WRF', '88-WRH', '88-WRJ', '88-WRK', '94ET-05', '94ET-06', 'CB08-01', 'CMA08-09', 'CV08-05', 'CVM08-10', 'NT-01', 'S85C-06', 'S85C-09', 'S86MT-01A-01B', 'S87-SI-05', 'SD79-16', 'SD84-101', 'SD84-44', 'SD84-45', 'SD85-51', 'SD85-60','85-YRZ']
#'82-QNE' doesn't contain useful stations
for file in files:
    print("-working on file %s"%file)
    file0 = open('Picks/%s_pick.txt'%file, 'r')
    lines = file0.readlines()
    df0 = []
    for line in lines[2:]:
        values = re.split(r'\s+', line.strip())
        df0.append(np.asarray(values, dtype='float'))
    df0 = np.stack(df0)
    dfpick  = pd.DataFrame(df0,columns=['ffid','sou_pos','srf_pos','chan','fb_pick'])
    #ffid: Shot ID, sou_pos: Source poisition (Station number), srf_pos: Reciever position (Station number), Chan: Recording channel, 
    #fb_pick: Picked time of the first p-wave arriving to the recoding station.  
    
    #dfpick.to_csv('Picks/%s_pick.csv' % file, index=False)
    
    file1 = open('RPS/%s.RPS'%file, 'r')
    lines = file1.readlines()
    df1   = []
    for line in lines[25:]:
        temp = re.split('\s+', line)[1:11]
        df1.append(np.asarray([temp[0],temp[3], temp[5], temp[6], temp[7]], dtype=float))
    df1 = np.stack(df1)
    dfRPS  = pd.DataFrame(df1,columns=['STATION','DATUM','EASTING','NORTHING','S_ELEVATION'])
    #STATION: Station number, DATUM: static correction datum, EASTING, NORTHING, S_ELEVATION: Station Elevation ASL. 
    
    print("-- file %s : dataframe ceated, now merging data"%file)
    
    # Joining the DataFrames based on 'sou_pos' and 'STATION'
    df_merged_sou = pd.merge(dfpick, dfRPS, left_on='sou_pos', right_on='STATION', how='left')

    # Renaming the columns
    df_merged_sou = df_merged_sou.rename(columns={'EASTING': 'S_X', 'NORTHING': 'S_Y', 'S_ELEVATION': 'S_Z'})

    # Joining the DataFrames based on 'srf_pos' and 'STATION'
    df_merged_srf = pd.merge(dfpick, dfRPS, left_on='srf_pos', right_on='STATION', how='left')

    # Renaming the columns
    df_merged_srf = df_merged_srf.rename(columns={'EASTING': 'R_X', 'NORTHING': 'R_Y', 'S_ELEVATION': 'R_Z'})

    # Selecting the desired columns
    df = dfpick[['sou_pos', 'srf_pos', 'fb_pick']].copy()

    # Adding the columns from df_merged_sou to df
    df[['S_X', 'S_Y', 'S_Z']] = df_merged_sou[['S_X', 'S_Y', 'S_Z']]

    # Adding the columns from df_merged_srf to df
    df[['R_X', 'R_Y', 'R_Z']] = df_merged_srf[['R_X', 'R_Y', 'R_Z']]
    print("--- file %s : data merging completed, now assigning picks"%file)

    # Dropping invalid picks
    df = df[df['fb_pick'] > 0]

    # Source reciever separation (offset)
    df['S_R_Separation'] = np.sqrt((df['S_X'] - df['R_X'])**2 + (df['S_Y'] - df['R_Y'])**2)

    # Assigning picks according to source-reciever geometry

    df['ABCD_t'] = np.where(df['sou_pos'] < df['srf_pos'], df['fb_pick'], -9999.00)
    df['GFED_t'] = np.where(df['sou_pos'] > df['srf_pos'], df['fb_pick'], -9999.00)

    # Source-reciever geometry labelling
    df['S_Geometry'] = np.where(df['ABCD_t'] > 0, 'A', 'G')

    # Separating picks according to geometry
    stations_A = df[df['S_Geometry'] == 'A']
    stations_G = df[df['S_Geometry'] == 'G']

    # Cleaning and labelling: Stations_A
    stations_A = stations_A.drop('GFED_t', axis=1)
    stations_A = stations_A.rename(columns={'S_R_Separation': 'AD_Separation'})
    stations_A = stations_A.drop(['fb_pick'], axis=1)

    # Cleaning and labelling Stations_G
    stations_G = stations_G.drop('ABCD_t', axis=1)
    stations_G = stations_G.drop(['R_X','R_Y','R_Z'], axis=1)
    stations_G = stations_G.drop(['fb_pick'], axis=1)
    stations_G = stations_G.rename(columns={'S_R_Separation': 'GD_Separation'})

    # Merging shots based on common reciever station.
    df = pd.merge(stations_A, stations_G, on='srf_pos', suffixes=('_A', '_G'))
    df = df.drop(['S_Geometry_A','S_Geometry_G'], axis=1)

    columns_order = ['srf_pos',
     'R_X',
     'R_Y',
     'R_Z',
     'sou_pos_A',
     'AD_Separation',
     'ABCD_t',
     'S_X_A',
     'S_Y_A',
     'S_Z_A',
     'sou_pos_G',
     'GD_Separation',
     'GFED_t',
     'S_X_G',
     'S_Y_G',
     'S_Z_G',]
    df=df[columns_order]
    
    print("--- file %s : picks assigned, now assigning ABFG"%file)
    
    # Keeping A and G with similar separation from D
    condition = (df['srf_pos'] - df['sou_pos_A']) == (df['sou_pos_G'] - df['srf_pos'])
    df = df.loc[condition].reset_index()   
    
    # Assigning ABFG values from where G was a reciever
    for i in range(len(df)):
        mask = (df['srf_pos'] == df['sou_pos_G'].iloc[i]) & (df['sou_pos_A'] == df['sou_pos_A'].iloc[i]) 
        if mask.any():
            df.at[i, 'ABFG_t'] = df.loc[mask, 'ABCD_t'].iloc[0]
            df.at[i, 'sou_pos_AG'] = df.loc[mask, 'sou_pos_A'].iloc[0]
            df.at[i, 'AG_Separation'] = df.loc[mask, 'AD_Separation'].iloc[0]
        else:
            df.at[i, 'ABFG_t'] = -9999.00

    print("---- file %s : exporting stations_D"%file)

    #df.to_csv('Stations_D/%s_stations_D.csv' % file, index=False)

    df = df.drop(df[df['ABFG_t'] <= 0].index) #Drops false picks
    print("----- file %s : data processing started"%file)

    #calculating T_plus time in seconds Yilmaz P378 (3 - 47 a)
    df['Tplus']=(df['ABCD_t']+df['GFED_t']-df['ABFG_t'])*0.001 
    #calculating T_minus time in seconds Yilmaz P378 (3 - 47 b)
    df['Tminus']=(df['ABCD_t']-df['GFED_t']+df['ABFG_t'])*0.001 

    # Assuming the weathering layer velocity as 800 m/s
    Vw = 800 
    # Yilmaz P379 (3 - 48 c) solved for calculating Vb (subweathering velocity)
    df['Vb']=(2*df['AD_Separation'])/(df['Tminus']-df['Tplus'])  
    # Yilmaz P378 (3 - 48 a) solved for Zw (depth to subweathering)
    df['Zw'] = (df['Tplus'] * df['Vb'] * Vw) / (2 * np.sqrt(df['Vb']**2 - Vw**2))

    #df.to_csv('%s_df_AGD.csv' % file, index=False)

    #1: constant station separation
    #2 and 3: Avoiding suspected picking issues
    #4 and 5: Avoiding subweathering velocities out of the accepted range
    # 6-9: avoiding too small or too large source reciever separation
    condition1 = (df['Tplus']) > 0
    condition2 = (df['Tminus']) > 0
    condition3 = (df['Vb'] >= 1600)
    condition4 = (df['Vb'] <= 2300)

    # Combine the conditions using logical AND
    conditions = condition1 & condition2 & condition3 & condition4

    # Filter the DataFrame using the conditions
    df = df.loc[conditions].reset_index()

    print("------ file %s : exporting results"%file)

    # Taking the mean of the estimates
    df_Zw_means = df.loc[:, ['srf_pos', 'R_X', 'R_Y', 'R_Z', 'Vb', 'Zw']]
    df_Zw_means = df_Zw_means.groupby('srf_pos').mean().reset_index()
    df_Zw_means['Transect_Distance']  = ((df_Zw_means['R_X']-df_Zw_means['R_X'][0])**2+(df_Zw_means['R_Y']-df_Zw_means['R_Y'][0])**2)**0.5
    df_Zw_means.to_csv('Zw_means/%s_df_Zw_means.csv' % file, index=False)

    easting = df_Zw_means['R_X']
    northing = df_Zw_means['R_Y']
    depth = df_Zw_means['Zw']

    # Increase the figure size in the x-axis
    fig, ax = plt.subplots(figsize=(12, 6))

    # Create scatter plot with colormap
    scatter = ax.scatter(easting, northing, c=depth, cmap='jet', s=1)

    # Add colorbar using the scatter plot as the mappable
    cbar = plt.colorbar(scatter)
    cbar.set_label('Estimated depth (m)')

    # Add axis labels and title
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    ax.set_title('Depth to the base of the weathering layer')

    # Disable scientific notation on the y-axis tick labels
    formatter = ticker.ScalarFormatter(useOffset=False)
    ax.yaxis.set_major_formatter(formatter)
    ax.xaxis.set_major_formatter(formatter)

    # Save the plot
    plt.savefig('Transects_top/Estimated_Depths_%s.png' % file, dpi=300)
    # Show the plot
    #plt.show()
    plt.close()
    
    fig, axs = plt.subplots(2, 1, figsize=(10, 8))  # Adjust the figure size as needed
    
    # Plot surface and bedrock elevation
    axs[0].plot(df_Zw_means['Transect_Distance'], df_Zw_means['R_Z'], label='Surface')
    axs[0].plot(df_Zw_means['Transect_Distance'], df_Zw_means['R_Z'] - df_Zw_means['Zw'], label='Bedrock')
    axs[0].legend()
    axs[0].set_xlabel('Along-transect distance (m)')
    axs[0].set_ylabel('Elevation ASL (m)')
    axs[0].set_xlim(df_Zw_means['Transect_Distance'].min() - 10, df_Zw_means['Transect_Distance'].max() + 10)  # Extend X-axis by 10 units on both sides
    
    # Plot weathered zone thickness
    axs[1].plot(df_Zw_means['Transect_Distance'], df_Zw_means['Zw'], 'r', label='Weathered zone')
    axs[1].legend()
    axs[1].set_xlabel('Along-transect distance (m)')
    axs[1].set_ylabel('Thickness (m)')
    axs[1].set_xlim(df_Zw_means['Transect_Distance'].min() - 10, df_Zw_means['Transect_Distance'].max() + 10)  # Extend X-axis by 10 units on both sides
    
    plt.tight_layout()  # Adjust subplot spacing
    
    plt.subplots_adjust(top=0.9)  # Increase the top spacing for the title
    plt.suptitle('Transect Side', fontsize=16, y=0.98)  # Add the title above the subplots
    
    plt.savefig('Transects_side/%s_transect_side.png' % file, dpi=300)
    #plt.show()
    plt.close()


    
    

In [ ]:
#Part 1.2
df_Zw_means = []
for file in files:
    df = pd.read_csv('Zw_means/%s_df_Zw_means.csv' % file)
    df = df.assign(File=file)  # Create a new column 'File' and assign the file name to it
    df_Zw_means.append(df)

df_Zw_means = pd.concat(df_Zw_means)
df_Zw_means.to_csv('Estimated_Depths.csv', index=False)

In [ ]:
#Part 2: reading and plotting the results.

In [ ]:
#Part 2.1: Reading the results
df_Zw_means = pd.read_csv('Estimated_Depths.csv') 

In [ ]:
#Part 2.2: Map plot

# Load the map image
map_image = mpimg.imread('map_for_plot.png')

# Scatter plot with colormap
fig, ax = plt.subplots(figsize=(12, 6))


scatter = ax.scatter(df_Zw_means['R_X'], df_Zw_means['R_Y'], c=df_Zw_means['Zw'], cmap='jet', s=1, vmin=0, vmax=100)

x_min = df_Zw_means['R_X'].min() - 3000
x_max = df_Zw_means['R_X'].max() + 3000
y_min = df_Zw_means['R_Y'].min() - 3000
y_max = df_Zw_means['R_Y'].max() + 3000

ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)

# Set equal aspect ratio
aspect_ratio = abs((x_max - x_min) / (y_max - y_min))
ax.set_aspect(aspect_ratio)

# Disable scientific notation on the y-axis tick labels
formatter = ticker.ScalarFormatter(useOffset=False)
ax.yaxis.set_major_formatter(formatter)
ax.xaxis.set_major_formatter(formatter)

# Set the map image as the background using ax.imshow()
ax.imshow(map_image, extent=[x_min, x_max, y_min, y_max])

# Colorbar settings
cbar = plt.colorbar(scatter)
cbar.set_label('Zw (m)')

# Axis labels and title
ax.set_xlabel('Easting')
ax.set_ylabel('Northing')
ax.set_title('Depth to the Base of the Weathering Layer')

# Save the plot
plt.savefig('Estimated_Depths.png', dpi=300)

# Show the plot
plt.show()
plt.close()

In [ ]:
# Part 2.3.1 Access certain parts of seismic lines
srf_pos_min = 2863
srf_pos_max = 2887
desired_file = 'NT-01'

# Filter the data based on the srf_pos range and file name
filtered_data = df_Zw_means[(df_Zw_means['srf_pos'] >= srf_pos_min) & (df_Zw_means['srf_pos'] <= srf_pos_max) & (df_Zw_means['File'] == desired_file)]

# Check if there is any data matching the filter
if filtered_data.empty:
    print('No data')
else:
    # Create the figure and axes
    fig, axs = plt.subplots(2, 1, figsize=(10, 8))

    # Plot surface and bedrock elevation
    axs[0].plot(filtered_data['Transect_Distance'], filtered_data['R_Z'], label='Surface')
    axs[0].plot(filtered_data['Transect_Distance'], filtered_data['R_Z'] - filtered_data['Zw'], label='Bedrock')
    axs[0].legend()
    axs[0].set_xlabel('Along-transect distance (m)')
    axs[0].set_ylabel('Elevation ASL (m)')
    axs[0].set_xlim(filtered_data['Transect_Distance'].min() - 10, filtered_data['Transect_Distance'].max() + 10)

    # Plot weathered zone thickness
    axs[1].plot(filtered_data['Transect_Distance'], filtered_data['Zw'], 'r', label='Weathered zone')
    axs[1].legend()
    axs[1].set_xlabel('Along-transect distance (m)')
    axs[1].set_ylabel('Thickness (m)')
    axs[1].set_xlim(filtered_data['Transect_Distance'].min() - 10, filtered_data['Transect_Distance'].max() + 10)

    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.suptitle('Transect Side: Part of the transect', fontsize=16, y=0.98)

    # Save the plot
    plt.savefig('%s_transect_side_partial.png' % desired_file, dpi=300)
    plt.show()
    plt.close()


In [ ]:
#Part 2.3.2 Examples plot
# Define the desired srf_pos range and file name for the first plot
srf_pos_min_1 = 2863
srf_pos_max_1 = 2887
desired_file_1 = 'NT-01'

# Filter the data for the first plot
filtered_data_1 = df_Zw_means[(df_Zw_means['srf_pos'] >= srf_pos_min_1) & (df_Zw_means['srf_pos'] <= srf_pos_max_1) & (df_Zw_means['File'] == desired_file_1)]

# Define the desired srf_pos range and file name for the second plot
srf_pos_min_2 = 1211
srf_pos_max_2 = 1235
desired_file_2 = 'CV08-05'

# Filter the data for the second plot
filtered_data_2 = df_Zw_means[(df_Zw_means['srf_pos'] >= srf_pos_min_2) & (df_Zw_means['srf_pos'] <= srf_pos_max_2) & (df_Zw_means['File'] == desired_file_2)]

# Check if there is any data matching the filter for each plot
if filtered_data_1.empty:
    print('No data - file1')
elif filtered_data_2.empty:
    print('No data - file 2')
else:
    # Create the figure and axes for the subplots
    fig, axs = plt.subplots(1, 2, figsize=(14, 6))

    # Settings for the first plot
    ax1 = axs[0]
    ax1.plot(filtered_data_1['Transect_Distance'], filtered_data_1['R_Z'], label='Surface')
    ax1.plot(filtered_data_1['Transect_Distance'], filtered_data_1['R_Z'] - filtered_data_1['Zw'], label='Bedrock')
    ax1.legend(loc= 'center right')
    ax1.set_xlabel('Along-transect distance (m)')
    ax1.set_ylabel('Elevation ASL (m)')
    ax1.set_xlim(filtered_data_1['Transect_Distance'].min() - 10, filtered_data_1['Transect_Distance'].max() + 10)
    ax1.set_title('Transect Side: Across a Dune Strike')  
    # Print coordinates
    ax1.plot(filtered_data_1['Transect_Distance'].iloc[[0, -1]], filtered_data_1['R_Z'].iloc[[0, -1]], 'kx', markersize=5)
    ax1.text(filtered_data_1['Transect_Distance'].iloc[0], filtered_data_1['R_Z'].iloc[0], f"{filtered_data_1['R_X'].iloc[0]}, {filtered_data_1['R_Y'].iloc[0]}", ha='left', va='bottom', fontsize=8)
    ax1.text(filtered_data_1['Transect_Distance'].iloc[-1], filtered_data_1['R_Z'].iloc[-1], f"{filtered_data_1['R_X'].iloc[-1]}, {filtered_data_1['R_Y'].iloc[-1]}", ha='right', va='top', fontsize=8)

    # Settings for the second plot
    ax2 = axs[1]
    ax2.plot(filtered_data_2['Transect_Distance'], filtered_data_2['R_Z'], label='Surface')
    ax2.plot(filtered_data_2['Transect_Distance'], filtered_data_2['R_Z'] - filtered_data_2['Zw'], label='Bedrock')
    ax2.legend(loc= 'center right')
    ax2.set_xlabel('Along-transect distance (m)')
    ax2.set_ylabel('Elevation ASL (m)')
    ax2.set_xlim(filtered_data_2['Transect_Distance'].min() - 10, filtered_data_2['Transect_Distance'].max() + 10)
    ax2.set_title('Transect Side: Along a Dune Strike')
    # Print coordinates 
    ax2.plot(filtered_data_2['Transect_Distance'].iloc[[0, -1]], filtered_data_2['R_Z'].iloc[[0, -1]], 'kx', markersize=5)
    ax2.text(filtered_data_2['Transect_Distance'].iloc[0], filtered_data_2['R_Z'].iloc[0], f"{filtered_data_2['R_X'].iloc[0]}, {filtered_data_2['R_Y'].iloc[0]}", ha='left', va='bottom', fontsize=8)
    ax2.text(filtered_data_2['Transect_Distance'].iloc[-1], filtered_data_2['R_Z'].iloc[-1], f"{filtered_data_2['R_X'].iloc[-1]}, {filtered_data_2['R_Y'].iloc[-1]}", ha='right', va='top', fontsize=8)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)

    # Save the plot
    plt.savefig('transect_panel.png', dpi=300)
    plt.show()
    plt.close()


In [ ]:
# Part 2.4 Zw histogram
plt.hist(df_Zw_means['Zw'], bins=50,log=True)  # 'bins' parameter sets the number of bins in the histogram

# Adding labels and title
plt.xlabel('Zw')
plt.ylabel('Frequency')
plt.title('Depths Histogram')

#Saving the histogram figure
plt.savefig('histogram.png', dpi=300)

# Displaying the histogram
plt.show()
plt.close()

In [ ]:
#Part 2.5: Zw and Vb averages

df_Zw_means['Zw'].mean(), df_Zw_means['Vb'].mean()


In [ ]:
#Part 2.6 The plus minus method diagram

# Define the coordinates of the stations and points
A = (0, 0)
D = (150, 0)
G = (300, 0)
B = (30, -20)
C = (120, -20)
E = (180, -20)
F = (270, -20)

# Create the figure and axes
fig, ax = plt.subplots(figsize=(7, 4))

# Set up the axes for ABCD raypath subplot
ax.set_xlim(-10, 310)
ax.set_ylim(-22.5, 8)

# Draw the surface and refractor lines for ABCD raypath
ax.axhline(y=0, color='black', linestyle='--', label='Surface')
ax.axhline(y=-20, color='black', linestyle='-', label='Refractor')

# Draw a vertical line from point D to the refractor with dash-dot line style
ax.plot([D[0], D[0]], [D[1], -20], linestyle='-.', color='black', label='Zw')

# Plot line 1 extending from A to B, B to C, and C to D for ABCD raypath
ax.plot([A[0], B[0], C[0], D[0]], [A[1], B[1], C[1], D[1]], 'r-', label='ABCD raypath')
ax.plot([G[0], F[0], E[0], D[0]], [G[1], F[1], E[1], D[1]], 'b-', label='GFED raypath')
ax.plot([A[0], B[0], F[0], G[0]], [A[1], B[1], F[1], G[1]], 'yellow', linestyle='dashdot', label='ABFG raypath')

# Add labels for the points in ABCD raypath subplot
ax.text(*A, 'A', ha='center', va='bottom', fontsize=12)
ax.text(*D, 'D', ha='center', va='bottom', fontsize=12)
ax.text(*G, 'G', ha='center', va='bottom', fontsize=12)
ax.text(*B, 'B', ha='center', va='top', fontsize=12)
ax.text(*C, 'C', ha='center', va='top', fontsize=12)
ax.text(*E, 'E', ha='center', va='top', fontsize=12)
ax.text(*F, 'F', ha='center', va='top', fontsize=12)

# Remove ticks and labels for x and y axes
ax.set_xticks([])
ax.set_yticks([])

#legends settings
ax.legend(loc='upper right', fontsize=12, frameon=False, shadow=True,  ncol=3)

# Set the title
ax.set_title('The Plus Minus Method', fontsize=16, fontweight='bold')

# Save the plot
plt.savefig('The_Plus_Minus_Method_Single_Panel.png', dpi=300)

# Display the plot
plt.show()
plt.close()